In [2]:
import numpy as np

In [4]:
def loadExData():
    return [[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

data = loadExData()
u, sigma, vt = np.linalg.svd(data)
print("奇异值：", sigma)
# 重构数据
data_ = u[:, :3]*np.mat([[sigma[0], 0, 0], [0, sigma[1], 0], [0, 0, sigma[2]]])*vt[:3, :]
print("---------------------------------------------------------------------------------------")
print("重构数据：",data_)

奇异值： [9.72140007e+00 5.29397912e+00 6.84226362e-01 1.50962387e-15
 1.15387192e-31]
---------------------------------------------------------------------------------------
重构数据： [[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 -2.84366098e-16
  -2.94015497e-16]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00  4.47489534e-16
   4.28190736e-16]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  3.09573758e-16
   2.99924358e-16]
 [ 5.00000000e+00  5.00000000e+00  5.00000000e+00 -1.47703573e-16
  -1.95842150e-16]
 [ 1.00000000e+00  1.00000000e+00 -5.70229711e-16  2.00000000e+00
   2.00000000e+00]
 [-7.49390630e-17  9.96896569e-16 -1.34350906e-15  3.00000000e+00
   3.00000000e+00]
 [-8.18314124e-17  2.75447132e-16 -3.13743829e-16  1.00000000e+00
   1.00000000e+00]]


In [3]:
# 三种相似度度量方法
# 欧式距离
def ecludSim(inA,inB):
    return 1.0/(1.0 + np.linalg.norm(inA - inB))   # 距离越大相似度越小，距离为0时相似度最大，为1


# 皮尔逊相关系数，也就是概率论书本讲到的那个相关系数
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*np.corrcoef(inA, inB, rowvar = 0)[0][1]


# 余弦相似度计算
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = np.linalg.norm(inA)*np.linalg.norm(inB)
    return 0.5+0.5*(num/denom)

In [7]:
# 餐馆菜肴推荐引擎、推荐未尝过的菜肴
# ###############################################################
def loadExData():
    return [[4, 4, 0, 2, 2],
            [4, 0, 0, 3, 3],
            [4, 0, 0, 1, 1],
            [1, 1, 1, 2, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0]]

# 计算用户对物品的评分值估计
def standEst(dataMat, user, simMeas, item):
    n = np.shape(dataMat)[1]  # 列，基于物品的
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):  # 遍历每个菜肴
        userRating = dataMat[user,j]
        if userRating == 0:  # 没尝过的忽略，不用计算
            continue
        # 返回均吃过两个不同菜肴的用户评分索引
        overLap = np.nonzero(np.logical_and(dataMat[:, item].A>0,
                                      dataMat[:, j].A>0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            # 计算菜肴评分相似度作为给定用户对未尝过菜肴的估计评分值
            similarity = simMeas(dataMat[overLap,item],
                                   dataMat[overLap,j])
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal  # 使得估计值分数规范化到0-5

# # 基于SVD的评分估计
# def standEst_(dataMat, user, simMeas, item):
#     n = np.shape(dataMat)[1]  # 列，基于物品的
#     simTotal = 0.0; ratSimTotal = 0.0
#     u, sigma, vt = np.linalg.svd(data)
#     sig3 = np.eye(3)*sigma[:3]
#     data_ = u[:, :3]*sig3*vt[:3, :]
#     for j in range(n):  # 遍历每个菜肴
#         userRating = dataMat[user,j]
#         if userRating == 0:  # 没尝过的忽略，不用计算
#             continue
#         # 返回均吃过两个不同菜肴的用户评分索引
#         overLap = np.nonzero(np.logical_and(data_[:, item].A>0,
#                                       data_[:, j].A>0))[0]
#         if len(overLap) == 0:
#             similarity = 0
#         else:
#             # 用菜肴评分相似度来计算给定用户对未尝过菜肴的估计评分值
#             similarity = simMeas(dataMat[overLap,item],
#                                    dataMat[overLap,j])
#         print('the %d and %d similarity is: %f' % (item, j, similarity))
#         simTotal += similarity
#         ratSimTotal += similarity * userRating  # 还考虑相似度和当前用户评分的乘积
#     if simTotal == 0: return 0
#     else: return ratSimTotal/simTotal  # 使得估计值分数规范化到0-5


def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    """""
    Parameters:
          dataMat: 数据
          user: 给定用户  
          N: 取最高的前几个推荐结果
          simMeas: 相似度度量方法
          estMethod: 评分值估计函数方法
    Returns:
        一个list，保存有最高的前几个推荐结果
    """""
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]  # 返回未尝过菜肴的索引
    # 若全部品尝过则函数结束，所谓推荐肯定是没尝过的~~
    if len(unratedItems) == 0:
        return 'you rated everything'
    itemScores = []  # 用于保存结果
    for item in unratedItems:  # 遍历没吃过的菜肴
        estimatedScore = estMethod(dataMat, user, simMeas, item)  # 计算菜肴估计值
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]


data = loadExData()
data = np.mat(data)
out = recommend(data, 2)
print(out)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000
[(2, 2.5), (1, 2.0243290220056256)]


##### 构建推荐引擎所面临的挑战：
##### 效率问题：
1、不需要每次运行程序都要计算一次SVD
2、数据中大部分为0，可以通过非零元素来节省内存和计算开销
3、各个物品的相似度值可以计算保存起来重复使用
##### 冷启动问题：
如何在缺乏数据的情况下更好的推荐，也就是说，在协同过滤杨景下，由于新物品到来时由于缺乏所有用户对其的喜好信息，因此无法判断每个用户对其的喜好。而无法判断某个用户对其的喜好，也就无法利用该商品.解决办法：将推荐看成是搜索问题，对物品进行扩展，通过各种标签来描述物品，比如价格贵，荤菜等，这样即使没用用户对某个物品进行评价，也能找到共通的信息.

In [6]:
# 基于SVD的图像压缩
def printMat(inMat, thresh=0.8):
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k]) > thresh:
                print(1, end="")
            else: print(0, end="")
        print('')

def imgCompress(numSV=2, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = np.mat(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    U,Sigma,VT = np.linalg.svd(myMat) #奇异值分解
    SigRecon = np.mat(np.zeros((numSV, numSV)))
    for k in range(numSV):#construct diagonal matrix from vector
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:] #重构,只要numSV维
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

imgCompress() 
# 仅仅用两个奇异值就可以精确的重构数据，但所需要的矩阵元素信息只有原来的十分之一，64+64+2/32*32

****original matrix******
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
00000000000111111